#Model Hub Login config (ToDO)

Import /install libraries

In [1]:
!pip install datasets
!pip install tokenizers
!pip install transformers[torch]
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.0 MB/s eta 0:00:00


# Data sets
Load data from CSV files for test and dev as well as surpisal sorted csv file for train

custom dataset class? https://www.youtube.com/watch?v=PXOzkkB5eH0

In [3]:
import datasets
from datasets import load_dataset, DatasetDict
def create_ds_dict(ds_dir):
  '''args:str ds_dir: fpath to directory with sentence CSV files
          str train_type: rand or curriculum
          lst int seeds: list of seeds for shuffling dataset splits
     return DatasetDict ds_dict where {k-->ds_split:v-->ds with sentIDs and sentences}'''
  datFiles={"train":(ds_dir+"babylm_sent_train.csv"),"validation":(ds_dir+"babylm_sent_dev.csv"),"test":(ds_dir+"babylm_sent_test.csv")}
  csv_dict=load_dataset("csv", data_files=datFiles, sep="\037") #load train test dev split into a Datasetdictionary
  ds_dict=DatasetDict()
  for k,v in csv_dict.column_names.items(): #remove unecessary columns
    ds_dict=csv_dict.remove_columns([col for col in v if col!='sentid'and col!='sentence'])
  #shuffle data , according to train_type
  '''if train_type=="rand":
    print("pre-shuffle train:",ds_dict["train"]["sentence"][1])
    ds_dict["train"].shuffle(seed)
    print("post-shuffle train",ds_dict["train"]["sentence"][1])
  print("pre-shuffle valid:",ds_dict["validation"]["sentence"][1])
  ds_dict["validation"].shuffle(seed)
  print("post-shuffle valid:",ds_dict["validation"]["sentence"][1])
  print("pre-shuffle test:",ds_dict["test"]["sentence"][1])
  ds_dict["test"].shuffle(seed)
  print("post-shuffle test:",ds_dict["test"]["sentence"][1])'''
  return ds_dict


##Test ds_dict creation

In [4]:
ds_dir=""
rawDS=create_ds_dict(ds_dir)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1e2631a4080d9759/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
print(rawDS)
print([rawDS.column_names["train"]])#print([col for col in rawDS['train'].column_names if col!='sentid'and col!='sentence'])
print(rawDS['train'])
print(rawDS['validation'])
print(rawDS['test'])

DatasetDict({
    train: Dataset({
        features: ['sentid', 'sentence'],
        num_rows: 918982
    })
    validation: Dataset({
        features: ['sentid', 'sentence'],
        num_rows: 883051
    })
    test: Dataset({
        features: ['sentid', 'sentence'],
        num_rows: 909795
    })
})
[['sentid', 'sentence']]
Dataset({
    features: ['sentid', 'sentence'],
    num_rows: 918982
})
Dataset({
    features: ['sentid', 'sentence'],
    num_rows: 883051
})
Dataset({
    features: ['sentid', 'sentence'],
    num_rows: 909795
})


#Tokenizer

###Loading pre-trained BPE

In [ ]:
# code for loading tokenizer from pytorch object
tokenizer=torch.load("path to tokenizer") #likely wont work
#Code for loading tokenizer from HF wrapper (more likely to work)
tokenizer = AutoTokenizer.from_pretrained("path to tokenizer") #Could specify path to a checkpoint or a tokenizer directory (requires that tokenizer was saved using HF)

### Tokenizer components, hugging face links for component options and , GPT 2 BPE specification
1. [Normalization](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.normalizers) ( any cleanup of the text that is deemed necessary, such as removing spaces or accents, Unicode normalization, etc.):
  *   GPT2 BPE does not use a normalizer
2. [Pre-tokenization](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.pre_tokenizers) (splitting the input into words):
  *   GPT2 BPE is not dependent on significant  pre-tokenization
3. Running the input through the [model](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.models) (using the pre-tokenized words to produce a sequence of tokens) and train tokenizer using model specific [trainer](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.trainers):
  *   GPT2 BPE uses the BPE model
4. [Post-processing](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.processors) (adding the special tokens of the tokenizer, generating the attention mask and token type IDs):
  *   GPT2 BPE uses????
5. [Decoding](https://huggingface.co/docs/tokenizers/python/latest/components.html#decoders) (technically not a compondent of tokenization , but the process through which one decodes the output of tokenization):
  *   Logically, given a byte level encoder , should use a byte level decoder


In [6]:
#tokenizer related libraries
import tokenizers
from tokenizers import (
    normalizers, #1
    Tokenizer,
    pre_tokenizers, #2
    models, #3
    trainers,#3
    processors, #4
    decoders, #5
)
from transformers import PreTrainedTokenizerFast
def create_BPE_tok(sents):
  '''args: str list sents
     returns: PreTrainedTokenizerFast wrapped tokenizer'''
  tokenizer = Tokenizer(models.BPE()) #tokenizer objects are intialized with a model argument which specifies with algorithm they will use for tokenization
  #GPT-2 does not use a normalizer, so we skip #1 and go directly to #2: pre-tokenization
  tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False) #prevents default addition of space char at the beginning of a sentence
  #3 passing input to model and training tokenizer
  trainer = trainers.BpeTrainer(vocab_size=50272, special_tokens=["<|endoftext|>", '<image>', '</c>', '<PERSON>']) # EOS is apparently the only special token for BPE (rest based on baseline github) , set vocab size according to babyLM baseline model config val
  tokenizer.train_from_iterator(sents, trainer=trainer)#tokenizer.train(["train.txt"], trainer=trainer) #can use train from iterator to file by file
  tokenizer.post_processor = processors.ByteLevel(trim_offsets=False) #4
  tokenizer.decoder = decoders.ByteLevel()#5

  ###Save (and Push?) Tokenizer
  wrapped_tokenizer = PreTrainedTokenizerFast(
      tokenizer_object=tokenizer,
      bos_token="<|endoftext|>",
      eos_token="<|endoftext|>",
  )
  #usable by HF to recreate this tokenizer
  wrapped_tokenizer.save_pretrained("babyLM10M-byte-level-BPE-tokenizer")
  #tokenizer.push_to_hub("babyLM10M-byte-level-BPE-tokenizer") #usable if logged into HF hub
  return wrapped_tokenizer

##Test:Tokenizer

In [7]:
import random
random.seed(32)
train=rawDS["train"]["sentence"]
print(train[1])
random.shuffle(train)
print(train[1])
tokenizer=create_BPE_tok(train)

kee .
And then Cannata and Doss order a hit... 'cause she snuffed out their old friend.


In [15]:
#can test either tokenizer trained in notebook , or loaded tokenizer,
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)
print(encoding)
#print(encoding.tokens)
print(tokenizer.decode(encoding))
print(tokenizer(sentence))
print(tokenizer.tokenize(sentence))
triSent=train[-3:]
print(triSent)



[1020, 260, 2228, 333, 18114, 11408, 17]
Let's test this tokenizer.
{'input_ids': [1020, 260, 2228, 333, 18114, 11408, 17], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
['Let', "'s", 'Ġtest', 'Ġthis', 'Ġtoken', 'izer', '.']
['Oh dear!', 'but an interest of thought. In the distinction of the various forms of', 'So, along with single tooth replacement, you get differentiation of the tooth rap , so that you get canines and incisors and molars and all sorts of different nicely  and complicated teeth.']


In [17]:
triBatch=tokenizer(triSent,max_length=8,truncation=False,return_overflowing_tokens=False,return_length=False)
triBatchTrunc=tokenizer(triSent,max_length=8,truncation=True,return_overflowing_tokens=False,return_length=True)
triBatchTruncOver=tokenizer(triSent,max_length=8,truncation=True,return_overflowing_tokens=True,return_length=True)

print("triBatch")
print(triBatch)
print(triBatch.keys())
for id in triBatch["input_ids"]:
  print(tokenizer.decode(id))

print("triBatchTrunc")
print(triBatchTrunc)
print(triBatchTrunc.keys())
print(triBatchTrunc.length)
for id in triBatchTrunc["input_ids"]:
  print(tokenizer.decode(id))

print("triBatchTruncOver")
print(triBatchTruncOver)
print(triBatchTruncOver.keys())
print(triBatchTruncOver.length)
for id in triBatchTruncOver["input_ids"]:
  print(tokenizer.decode(id))

triBatch
{'input_ids': [[642, 2144, 4], [1163, 309, 1403, 236, 842, 17, 655, 210, 12488, 236, 210, 3083, 4633, 236], [570, 15, 1564, 310, 2124, 7535, 16052, 15, 243, 452, 43742, 236, 210, 7535, 4067, 346, 370, 267, 243, 452, 366, 1281, 241, 1252, 219, 799, 241, 12331, 950, 241, 375, 6844, 236, 1049, 12023, 170, 241, 6949, 4692, 17]], 'token_type_ids': [[0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
Oh dear!
but an interest of thought. In the distinction of the various forms of
So, along with single tooth replacement, you get differentiation of the tooth rap, so that you get canines and incisors and molar

# Get Batches (data loading)
1.   Order sentences:Shuffled during dataset laoding (depending on whether currOPT or randOPT)
2.   Tokenize and truncate sentences according to context length
3.   use data collator to take tokenized list and dynamically pad (pad to longest) as per batch size (32 sequences)


In [8]:
import torch
from transformers import DataCollatorForLanguageModeling
import csv
def dynamically_pad_pretokenized(tokenized_batch,tokenizer,log_file):
  '''arg: batch of inputIds and corresponding tokenizer
     log: info about tensor dimensions into a csv
     return: dynamically padded batch of pt tensors for causal LM'''
  tokenizer.pad_token = tokenizer.eos_token
  data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False , return_tensors="pt")
  tensor=data_collator(tokenized_batch)
  '''for k,v in tensor.items(): #prints information about tensors
    print(k,v.size())'''
   with open(log_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Key', 'Size 1', 'Size 2'])  # Write header row
        for k, v in tensor.items():
            writer.writerow([k, v.size(0), v.size(1)])  # Write key and tensor sizes
  return tensor

In [9]:
def create_tokenized_batches(sents,tokenizer,context_length,batch_size):
  '''Args: str list sents: ordered list of sentences to iterate over
           tokenizer
           context_length:
           batch_size
      return:array of pt tensors containing dynamically padded batches'''
  collated_batches=[]
  batch=[]
  for sent in sents:
    outputs = tokenizer(
        sent,
        truncation=True, #chunk sentence according to context length if too long
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    for input_id in outputs["input_ids"]:#input chunks count as seperate sequences
      if len(batch)<batch_size:
        batch.append(input_id)
      else: #batch_size reached ,  collate into tensor
        #print(len(batch))
        collated_batch=dynamically_pad_pretokenized(batch,tokenizer)
        #print(len(collated_batch))
        collated_batches.append(collated_batch)
        #print(len(collated_batches))
        batch=[]
        batch.append(input_id)
  print("batches created:",len(collated_batches))
  return collated_batches

## Test: Batching

In [10]:
trainDL=create_tokenized_batches(train,tokenizer,128,32)

Streaming output truncated to the last 5000 lines.
labels torch.Size([32, 71])
input_ids torch.Size([32, 37])
labels torch.Size([32, 37])
input_ids torch.Size([32, 128])
labels torch.Size([32, 128])
input_ids torch.Size([32, 128])
labels torch.Size([32, 128])
input_ids torch.Size([32, 54])
labels torch.Size([32, 54])
input_ids torch.Size([32, 38])
labels torch.Size([32, 38])
input_ids torch.Size([32, 111])
labels torch.Size([32, 111])
input_ids torch.Size([32, 92])
labels torch.Size([32, 92])
input_ids torch.Size([32, 128])
labels torch.Size([32, 128])
input_ids torch.Size([32, 94])
labels torch.Size([32, 94])
input_ids torch.Size([32, 100])
labels torch.Size([32, 100])
input_ids torch.Size([32, 56])
labels torch.Size([32, 56])
input_ids torch.Size([32, 53])
labels torch.Size([32, 53])
input_ids torch.Size([32, 54])
labels torch.Size([32, 54])
input_ids torch.Size([32, 111])
labels torch.Size([32, 111])
input_ids torch.Size([32, 72])
labels torch.Size([32, 72])
input_ids torch.Size([32

In [65]:
tensorDict=trainDL[1]
print(tensorDict.keys())
'''input_ids=tensorDict["input_ids"]
print(input_ids)
print(input_ids[1])
print(tokenizer.decode(input_ids[1]))'''
for k,v in tensorDict.items():
  print(k)
  print(v.shape)
  print(v.size)
  print(v)
  print(v[1])
  print(tokenizer.decode(v[1])) #can't decode labels???

dict_keys(['input_ids', 'labels'])
input_ids
torch.Size([32, 128])
<built-in method size of Tensor object at 0x7f5f17d0f100>
tensor([[ 344,  359,  370,  ...,    0,    0,    0],
        [ 745,  265,  210,  ...,    0,    0,    0],
        [ 344,  707,  452,  ...,    0,    0,    0],
        ...,
        [  76,  738,  263,  ...,    0,    0,    0],
        [ 579,  210,  661,  ...,    0,    0,    0],
        [2726,  249,  440,  ...,    0,    0,    0]])
tensor([ 745,  265,  210,  656, 3399,  249, 7418,  233,   17,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  

OverflowError: ignored

In [11]:
valid=rawDS['validation']['sentence']
random.shuffle(valid)
valDL=create_tokenized_batches(valid,tokenizer,128,32)

Streaming output truncated to the last 5000 lines.
labels torch.Size([32, 128])
input_ids torch.Size([32, 27])
labels torch.Size([32, 27])
input_ids torch.Size([32, 34])
labels torch.Size([32, 34])
input_ids torch.Size([32, 32])
labels torch.Size([32, 32])
input_ids torch.Size([32, 128])
labels torch.Size([32, 128])
input_ids torch.Size([32, 86])
labels torch.Size([32, 86])
input_ids torch.Size([32, 77])
labels torch.Size([32, 77])
input_ids torch.Size([32, 56])
labels torch.Size([32, 56])
input_ids torch.Size([32, 29])
labels torch.Size([32, 29])
input_ids torch.Size([32, 123])
labels torch.Size([32, 123])
input_ids torch.Size([32, 79])
labels torch.Size([32, 79])
input_ids torch.Size([32, 53])
labels torch.Size([32, 53])
input_ids torch.Size([32, 23])
labels torch.Size([32, 23])
input_ids torch.Size([32, 91])
labels torch.Size([32, 91])
input_ids torch.Size([32, 52])
labels torch.Size([32, 52])
input_ids torch.Size([32, 56])
labels torch.Size([32, 56])
input_ids torch.Size([32, 64])


In [12]:
def create_dataloaders(ds_dict,tokenizer,context_length,batch_size):# should work
  dl_dict={}
  for k,v in ds_dict.column_names.items():
    corpus=ds_dict[k]['sentence']
    print("pre-shuffling",k,corpus[1])
    random.shuffle(corpus)
    print("post-shuffling",k,corpus[1])
    dl_dict[k]=create_tokenized_batches(corpus,tokenizer,context_length,batch_size)
  '''dl_dict={"train":create_tokenized_batches(ds_dict["train"]["sentence"]),
           "validation":create_tokenized_batches(ds_dict["validation"]["sentence"]),
           "test":create_tokenized_batches(ds_dict["test"]["sentence"])}'''
  return dl_dict

In [43]:
#datDL=create_dataloaders(rawDS,tokenizer,128,32)

pre-shuffling train kee .
post-shuffling train Our goal as a college is to hit $5,000 students in accordance with the chancellors charge that UNL's total enrollment should reach 30,000 by 2017.


KeyboardInterrupt: ignored

In [29]:
print(len(datDL["train"]))

0


###Early Stopping /Convergence
Look at LSTM defaults  , specify threshold?

#OPT 125M model

##Configuration and initialization
Intialize an OPT model to closely replicat OPT125M architecture , using details from [(Zhang et al., 2022) ](https://https://arxiv.org/pdf/2205.01068.pdf) and [babyLM OPT125M vaseline](https://huggingface.co/babylm/opt-125m-strict-small/blob/main/config.json). See further details on the 15 params at huggingface OPTconfig documentation


In [14]:
from transformers.models.opt import OPTConfig
from transformers.models.opt.modeling_opt import OPTForCausalLM
def get_OPT_CLM():
  #Set config params
  config=OPTConfig()
  print(config) #check defaults
  config.vocab_size=50272 #vocab size used in babylm baseline
  config.num_hidden_layers=768 #See dLayers , table 1 zhang et Al.
  config.num_hidden_layers=12 #See #L , table 1 zhang et Al.
  config.ffn_dim=3072 #see OPT125m baseline
  config.num_attention_heads=12 #See #H , table 1 zhang et Al.
  config.activation_function="relu"#see zhang et al
  config.max_position_embeddings=2048 #see OPT125m baseline
  config.do_layer_norm_before=True #see OPT125m baseline
  config.word_embed_proj_dim=768 #generally equal to hidden size , intialized accoring to OPT125M baseline
  config.dropout=0.1 # see Zhang et Al : "We use a dropout of 0.1 throughout, but we do not apply any dropout to embeddings."
  config.attention_dropout=0.0 #see OPT125M baseline
  config.init_std=0.02 #Intialized according to OPT125M baseline however potnetially signifcant differences from zhang et Al: “For weight initialization ….. using a normal distribution with zero mean and standard deviation of 0.006.”” Standard deviation for output layers are scaled by a 1.0/ √ 2L term where L is the total number of layers. ”
  config.use_cache=True #see OPT125M baseline
  config.enable_bias=True #see OPT125M baseline
  config.layer_norm_elementwise_affine=True #see OPT125M baseline
  print(config) #check updates
  #intialize OPTm model with a causalLM head for training
  OPTCLM=OPTForCausalLM(config)
  #check architecture
  print(OPTCLM)
  model_size = sum(t.numel() for t in OPTCLM.parameters())
  print(f"OPT size: {model_size/1000**2:.1f}M parameters")
  return OPTCLM

#Optimizer

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5,betas=(0.9,0.95),weight_decay=0.1) #Zhang et Al "We use an AdamW optimizer (Loshchilov andHutter, 2017) with (β1, β2) set to (0.9, 0.95), and weight decay of 0.1"

##Scheduler (TODO)

OPT paper:"We follow a linear learning
rate schedule, warming up from 0 to the maximum
learning rate over the first 2000 steps in OPT-175B,
or over 375M tokens in our smaller baselines, and
decaying down to 10% of the maximum LR over
300B tokens. A number of mid-flight changes
to LR were also required (see Section 2.5). Our
batch sizes range from 0.5M to 4M depending on
the model size (see Table 1) and is kept constant
throughout the course of training."

Platanios et al:
*   for large experiments with tranformers T=50K (see curriculum hyperparameters) (large dataset was WMT with 4.5M sentences)
*   Twarmup  "is set to 10,000 in these experiments" => 1/5 of steps
*    "This schedule was proposed in the original
Transformer paper (Vaswani et al., 2017)"

Vaswani et al., 2017:
* "We trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the
bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps
(3.5 days)."
* Also used linear learning rate "We used warmup_steps = 4000." => small fraction
*Also had the same large WMT 4.5M sentence dataset


In [15]:
from transformers import get_scheduler
def create_lr_scheduler(num_train_epochs,total_training_steps,optimizer):
  #update_steps_per_epoch = len(train_dataloader) #update every batch
  lr_scheduler = get_scheduler(
      name="linear", #zhang et Al. followed a linear learning rate schedule , midflight changes not relevant to 125M model and  our small dataset
      optimizer=optimizer,
      num_warmup_steps=int(total_training_steps*0.05), #TODO set as a fraction of training steps=> going with 5%
      num_training_steps=total_training_steps
   )
  return lr_scheduler

# Evaluation

https://huggingface.co/learn/nlp-course/chapter3/4?fw=pt#the-evaluation-loop
"Since we want to evaluate the model regularly on the validation set during training, let’s write a function for that as well. It just runs through the evaluation dataloader and gathers all the losses across processes:"

In [13]:
def evaluate(model,eval_dataloader,device):
  '''evaluate model to get loss and perplexity'''
  model.eval()
  losses = []
  for step, batch in enumerate(eval_dataloader):
     batch_on_device = {k: v.to(device) for k, v in batch.items()}
     with torch.no_grad():
          outputs = model(batch_on_device["input_ids"], labels=batch_on_device["input_ids"])
     losses.append(outputs.loss)#losses.append(accelerator.gather(outputs.loss))### change
  loss = torch.mean(torch.cat(losses))
  try:
      perplexity = torch.exp(loss)
  except OverflowError:
      perplexity = float("inf")
  return loss.item(), perplexity.item()

##Early Stopping
class adapted with minimal changes from : https://gist.github.com/stefanonardo/693d96ceb2f531fa05db530f3e21517d

In [ ]:
#import numpy as np

class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        '''params:str mode: Indicates whether to minimize ('min') or maximize ('max') the monitored metric.
                 float min_delta: The minimum change required to qualify as an improvement in the monitored metric.
                 int patience: The number of epochs to wait for an improvement before stopping the training process.
                 bool percentage: If True, min_delta is interpreted as a percentage change rather than an absolute value.'''
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        # If patience is set to 0, always consider the current metric as the best
        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        # If the metric is NaN, consider it as a signal to stop early
        if torch.isnan():# orignal code used numpy:np.isnan(metrics):
            return True

        # Check if the current metric is better than the best
        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        # If the number of bad epochs exceeds patience, stop early
        if self.num_bad_epochs >= self.patience:
            print('Terminating because of early stopping!')
            return True
        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('Mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                # Check if a is smaller than (best - min_delta)
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                # Check if a is larger than (best + min_delta)
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                # Check if a is smaller than (best - (best * min_delta / 100))
                self.is_better = lambda a, best: a < best - (best * min_delta / 100)
            if mode == 'max':
                # Check if a is larger than (best + (best * min_delta / 100))
                self.is_better = lambda a, best: a > best + (best * min_delta / 100)


In [ ]:
#Example
es = EarlyStopping(patience=5)


num_epochs = 100
for epoch in range(num_epochs):
      train_one_epoch(model, data_loader)  # train the model for one epoch.
      metric = eval(model, data_loader_dev)  # evalution on dev set.
      if es.step(metric):
          break  # early stop criterion is met, we can stop now


#Training loop

Simple training and val loops

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
def train_one_step(model,device,batch,optimizer,lr_scheduler):
  #Forward Pass
  batch_on_device = {k: v.to(device) for k, v in batch.items()}
  fpass_output=model(input_ids=batch_on_device["input_ids"],labels=batch_on_device["labels"]) # Forward pass: **batch ??
  logits = fpass_output.logits  #Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax):logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size))
  loss =fpass_output.loss #cross-entropy loss: loss (torch.FloatTensor of shape (1,), optional, returned when labels is provided) — Language modeling loss (for next-token prediction).
  #update logging vars
  curr_loss=loss.item()
  curr_lr=lr_scheduler.get_lr()
  #backward pass
  loss.backward(loss)
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip the gradients to prevent exploding gradients => used 1.0 to 0.3 clipping in OPT paper
  optimizer.step()
  lr_scheduler.step()
  optimizer.zero_grad()  # Clear gradients from the previous optimization step
  return curr_loss , curr_lr

In [ ]:
def train_one_epoch(epoch,train_dataloader,eval_dataloader,num_training_steps,completed_steps,early_stopper):
  for step, batch in tqdm(enumerate(train_dataloader, start=1), total=num_training_steps,spath): #iterate with progress bar where  'step'= current batch index, 'batch' contains the input data for the current batch , represent batch 0 as batch 1
    loss,lr=train_one_step(model,device,batch,optimizer,lr_scheduler)
    completed_steps += 1
    if (epoch*step) % (epoch*log_steps) == 0: #LOGGING progress metrics computed on train set every log_steps
            print(#Insert tensorboard logging here?
                {
                    "lr": lr,
                    "steps": completed_steps,
                    "loss/train": loss #add train ppl?
                }
            )

    if ((epoch*step) % (epoch*eval_steps)) == 0:# Perform model evaluation on validation data after every eval_steps, set eval_steps=len(train_dataloader) for per epoch logging (generalizes to currOPT)
            eval_loss, perplexity = evaluate(model,eval_dataloader)  # Evaluate the model's performance on the evaluation dataset
            print({"loss/eval": eval_loss, "perplexity": perplexity})  # Print evaluation metrics
            #####check pointing code######
            torch.save({
            'epoch': epoch,
            'completed_steps':completed_steps
            'model_state_dict': model.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'lr':lr
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': eval_loss,
            }, spath+str(completed_steps)+'.pt') #save to path +name+'step:'+completed_steps+'.pt'
            if early_stopper.step(eval_loss):
              return model ,True #converged
            model.train()  # Set the model back to training mode
  return model , False #will continue training unless last epoch

In [ ]:
def training_loop():
  for epoch in range(0,num_epochs+1):
    model,converged=train_one_epoch()
    if converged:
      return model #trigger saving function
  ####SAVE MODEL
  return model

Can remove Accelerator specific code from loop (TODO)

*  Implement early stopping / convergence??
*  Implement saving
*  Implement tensor

In [ ]:
from tqdm.notebook import tqdm  #progress bar visualization
#import torch.nn #for replacing accelerator code
#from torch.nn import torch.nn.utils
early_stopper= EarlyStopping(patience=3, mode='min')#think about threshold??
eval_steps = 5_000  # Number of steps after which to evaluate the model- modify to every epoch?

model.train()  # Set the model in training mode
completed_steps = 0

for epoch in range(1,num_train_epochs+1):  #Iterate over the training epochs Start from 1 , generalizes to currOPT and randOPT
    for step, batch in tqdm(enumerate(train_dataloader, start=1), total=num_training_steps): #iterate with progress bar where  'step'= current batch index, 'batch' contains the input data for the current batch , represent batch 0 as batch 1
        #FORWARD PASS
        batch_on_device = {k: v.to(device) for k, v in batch.items()}
        fpass_output=model(input_ids=batch_on_device["input_ids"],labels=batch_on_device["labels"]) # Forward pass: **batch ??
        logits = output.logits  #Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax):logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size))
        loss =output.loss #cross-entropy loss: loss (torch.FloatTensor of shape (1,), optional, returned when labels is provided) — Language modeling loss (for next-token prediction).

        if (epoch*step) % (epoch*log_steps) == 0: #LOGGING progress metrics computed on train set every log_steps
            print(#Insert tensorboard logging here?
                {
                    "lr": lr_scheduler.get_lr(),  # NEED TO DEFINE?
                    "steps": completed_steps+1,
                    "loss/train": loss.item() #add train ppl?
                }
            )
        #BACKWARD PASS and OPTIMIZATION
        loss.backward(loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip the gradients to prevent exploding gradients => used 1.0 to 0.3 clipping in OPT paper
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()  # Clear gradients from the previous optimization step
        completed_steps += 1

        if ((epoch*step) % (epoch*eval_steps)) == 0:# Perform model evaluation on validation data after a certain number of steps, set eval_steps=len(train_dataloader) for per epoch logging
            eval_loss, perplexity = evaluate(model,eval_dataloader)  # Evaluate the model's performance on the evaluation dataset
            print({"loss/eval": eval_loss, "perplexity": perplexity})  # Print evaluation metrics
            if early_stopper.step(eval_loss):
              break
            model.train()  # Set the model back to training mode
            #accelerator.wait_for_everyone()  # Synchronize processes using the accelerator
            ######Create Save Model:##########


Saving model (push to hub?)
State dictionaries in accellarator (save_state() , trainer.save_model())

1.   save at the end of each epoch for randOPT
      Early stopping or stop after 100 eopochs for randOPT
2.   save by sentences /steps for curriculumOPT
     Early stopping not applicable for curriculumOPT

TensorBoard

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # Writer will output to ./runs/ directory by default.

##Example Tensorboard logging in ;\loop
x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())

model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

def train_model(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train_model(10)
writer.flush()

# Test train loops

##Collab Nvidia T4 run

In [17]:
from tqdm.notebook import tqdm  #progress bar visualization
import torch
import torch.nn #for replacing accelerator code
import torch.nn.utils
from transformers import AdamW
num_train_epochs=1
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
# use previously created trainDL, validDL as dataloaders and tokenizer as tokenizer

batches_per_epoch=len(trainDL)
total_steps=num_train_epochs*batches_per_epoch #update gradient every epoch
model=get_OPT_CLM() #OPT125M with causal LM head for training
model.to(device)
#use to intialize LR scheduler and optimizer
optimizer = AdamW(model.parameters(), lr=5e-5,betas=(0.9,0.95),weight_decay=0.1) #Zhang et Al "We use an AdamW optimizer (Loshchilov andHutter, 2017) with (β1, β2) set to (0.9, 0.95), and weight decay of 0.1"
lr_scheduler=create_lr_scheduler(num_train_epochs,total_steps,optimizer)


#########training loop#############
eval_steps = batches_per_epoch  # Number of steps after which to evaluate the model- modify to every epoch?
log_steps=100
model.train()  # Set the model in training mode
completed_steps = 0

for epoch in range(1,num_train_epochs+1):  #Iterate over the training epochs Start from 1 , generalizes to currOPT and randOPT
    for step, batch in tqdm(enumerate(trainDL, start=1), total=total_steps): #iterate with progress bar where  'step'= current batch index, 'batch' contains the input data for the current batch , represent batch 0 as batch 1
        #FORWARD PASS
        batch_on_device = {k: v.to(device) for k, v in batch.items()}
        fpass_output=model(input_ids=batch_on_device["input_ids"],labels=batch_on_device["labels"]) # Forward pass: **batch ??
        logits = fpass_output.logits  #Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax):logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size))
        loss =fpass_output.loss #cross-entropy loss: loss (torch.FloatTensor of shape (1,), optional, returned when labels is provided) — Language modeling loss (for next-token prediction).

        if (epoch*step) % (epoch*log_steps) == 0: #LOGGING progress metrics computed on train set every log_steps
            print(#GET RID OF ACCELERATOR
                {
                    "lr": lr_scheduler.get_lr(),  # NEED TO DEFINE?
                    "steps": completed_steps,  # Total number of completed steps
                    "loss/train": loss.item()
                }
            )
        #BACKWARD PASS and OPTIMIZATION
        loss.backward(loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip the gradients to prevent exploding gradients => used 1.0 to 0.3 clipping in OPT paper
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()  # Clear gradients from the previous optimization step
        completed_steps += 1

        if ((epoch*step) % (epoch*eval_steps)) == 0:# Perform model evaluation on validation data after a certain number of steps, set eval_steps=len(train_dataloader) for per epoch logging
            eval_loss, perplexity = evaluate(model,valDL,device)  # Evaluate the model's performance on the evaluation dataset
            print({"loss/eval": eval_loss, "perplexity": perplexity})  # Print evaluation metrics
            model.train()  # Set the model back to training mode
  #save final model , compute test set PPL?

cuda
OPTConfig {
  "_remove_final_layer_norm": false,
  "activation_function": "relu",
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "transformers_version": "4.30.2",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

OPTConfig {
  "_remove_final_layer_norm": false,
  "activation_function": "relu",
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "op

  0%|          | 0/28936 [00:00<?, ?it/s]

{'lr': [3.423236514522822e-06], 'steps': 99, 'loss/train': 9.693499565124512}
{'lr': [6.8810511756569855e-06], 'steps': 199, 'loss/train': 9.19655990600586}
{'lr': [1.0338865836791148e-05], 'steps': 299, 'loss/train': 8.572277069091797}
{'lr': [1.3796680497925313e-05], 'steps': 399, 'loss/train': 8.208853721618652}
{'lr': [1.7254495159059474e-05], 'steps': 499, 'loss/train': 7.207711696624756}
{'lr': [2.071230982019364e-05], 'steps': 599, 'loss/train': 6.9140400886535645}
{'lr': [2.4170124481327804e-05], 'steps': 699, 'loss/train': 7.2245192527771}
{'lr': [2.7627939142461967e-05], 'steps': 799, 'loss/train': 6.971721172332764}
{'lr': [3.1085753803596127e-05], 'steps': 899, 'loss/train': 7.121711730957031}
{'lr': [3.454356846473029e-05], 'steps': 999, 'loss/train': 6.659900188446045}
{'lr': [3.800138312586445e-05], 'steps': 1099, 'loss/train': 6.054701805114746}
{'lr': [4.145919778699862e-05], 'steps': 1199, 'loss/train': 6.4073004722595215}
{'lr': [4.4917012448132786e-05], 'steps': 129

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 26>:44                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/utils/clip_grad.py:76 in clip_grad_norm_        │
│                                                                                                  │
│    73 │   clip_coef_clamped = torch.clamp(clip_coef, max=1.0)                                    │
│    74 │   for ((device, _), [grads]) in grouped_grads.items():                                   │
│    75 │   │   if (foreach is None or foreach) and _has_foreach_support(grads, device=device):    │
│ ❱  76 │   │   │   torch._foreach_mul_(grads, clip_coef_clamped.to(device))  # type: ignore[cal   │
│    77 │   │   elif foreach:                                                                      │
│    78 │   │   │   raise RuntimeError(f'foreach=True was passed, but can\'t use the foreach API   │
│    79 │   │   else:                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

##RCJH1 GPU memory testing

GPU helper functions

In [ ]:
import pynvml
from pynvml import *  # provides access to NVIDIA's NVML (NVIDIA Management Library).enables  querying and monitoring of NVIDIA GPU devices.


def print_gpu_utilization():
    nvmlInit()  # Initializing the NVML library
    handle = nvmlDeviceGetHandleByIndex(0)  # Obtaining a handle for the GPU device at index 0
    info = nvmlDeviceGetMemoryInfo(handle)  # Retrieving memory information for the GPU device
    print(f"GPU memory occupied: {info.used//1024**2} MB.")  # Printing the GPU memory occupied in megabytes


In [ ]:
print_gpu_utilization() #starting utilization

Effects of initializing OPT125M

In [ ]:
from tqdm.notebook import tqdm  #progress bar visualization
import torch
import torch.nn #for replacing accelerator code
import torch.nn.utils
from transformers import AdamW
num_train_epochs=1
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
# use previously created trainDL, validDL as dataloaders and tokenizer as tokenizer

batches_per_epoch=len(trainDL)
total_steps=num_train_epochs*batches_per_epoch #update gradient every epoch
model=get_OPT_CLM() #OPT125M with causal LM head for training
model.to(device)

In [ ]:
print_gpu_utilization() #after moving model to device

Effects of training loop

In [ ]:
#use to intialize LR scheduler and optimizer
optimizer = AdamW(model.parameters(), lr=5e-5,betas=(0.9,0.95),weight_decay=0.1) #Zhang et Al "We use an AdamW optimizer (Loshchilov andHutter, 2017) with (β1, β2) set to (0.9, 0.95), and weight decay of 0.1"
lr_scheduler=create_lr_scheduler(num_train_epochs,total_steps,optimizer)


#########training loop#############
eval_steps = batches_per_epoch  # Number of steps after which to evaluate the model- modify to every epoch?
log_steps=100
model.train()  # Set the model in training mode
completed_steps = 0

for epoch in range(1,num_train_epochs+1):  #Iterate over the training epochs Start from 1 , generalizes to currOPT and randOPT
    for step, batch in tqdm(enumerate(trainDL, start=1), total=total_steps): #iterate with progress bar where  'step'= current batch index, 'batch' contains the input data for the current batch , represent batch 0 as batch 1
        #FORWARD PASS
        batch_on_device = {k: v.to(device) for k, v in batch.items()}
        fpass_output=model(input_ids=batch_on_device["input_ids"],labels=batch_on_device["labels"]) # Forward pass: **batch ??
        logits = fpass_output.logits  #Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax):logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size))
        loss =fpass_output.loss #cross-entropy loss: loss (torch.FloatTensor of shape (1,), optional, returned when labels is provided) — Language modeling loss (for next-token prediction).

        if (epoch*step) % (epoch*log_steps) == 0: #LOGGING progress metrics computed on train set every log_steps
            print(#GET RID OF ACCELERATOR
                {
                    "lr": lr_scheduler.get_lr(),  # NEED TO DEFINE?
                    "steps": completed_steps,  # Total number of completed steps
                    "loss/train": loss.item()
                }
            )
            print_gpu_utilization()
        #BACKWARD PASS and OPTIMIZATION
        loss.backward(loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip the gradients to prevent exploding gradients => used 1.0 to 0.3 clipping in OPT paper
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()  # Clear gradients from the previous optimization step
        completed_steps += 1

        if ((epoch*step) % (epoch*eval_steps)) == 0:# Perform model evaluation on validation data after a certain number of steps, set eval_steps=len(train_dataloader) for per epoch logging
            eval_loss, perplexity = evaluate(model,valDL,device)  # Evaluate the model's performance on the evaluation dataset
            print({"loss/eval": eval_loss, "perplexity": perplexity})  # Print evaluation metrics
            model.train()  # Set the model back to training mode

In [ ]:
from transformers import AdamW
def trainOPT125M(dat_dir,save_dir,train_type,num_train_epochs):
  # load datasets and shuffle/order sentences
  ordered_ds_dict=create_ds_dict(dat_dir,train_type,seeds)
  corpus=ordered_ds_dict["train"]["sentence"]
  tokenizer=create_BPE_tok(corpus)# train BPE tokenizer on corpus
  ordered_dataloaders=create_dataloaders(ordered_ds_dict,tokenizer,context_length,batch_size)
  train_dataloader=ordered_dataloaders["train"]
  batches_per_epoch=len(train_dataloader)
  total_steps=num_train_epochs*batches_per_epoch #update gradient every epoch
  #use to intialize LR scheduler and optimizer
  optimizer = AdamW(model.parameters(), lr=5e-5,betas=(0.9,0.95),weight_decay=0.1) #Zhang et Al "We use an AdamW optimizer (Loshchilov andHutter, 2017) with (β1, β2) set to (0.9, 0.95), and weight decay of 0.1"
  lr_schedule=create_lr_scheduler(num_train_epochs,total_steps,optimizer)
  model=get_OPT_CLM() #OPT125M with causal LM head for training
  #training loop
  #save final model , compute test set PPL?
